# Big Data Modeling and Management Assigment - Homework 1

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_.

### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions


### Questions

1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers? [Hint: 3-node path]
    1. Which `Country` has the most reviews for its beers? [Hint: 5-node path]
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV? [Hint: database has NaN values]
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try? (Justify your answer well!) [Hint: database has NaN values]
1. Using Graph Algorithms answer **one** of the following questions: [Hint: make sure to clear the graph before using it again]
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them? [Please use limit of 1000 on beer-review-user path]]
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community? [Please use limit of 1000 on beer-review-user path]]
1. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`? [Hint:inspect two subsets: with and without the beer in question]
    2. Which user is the most influential when it comes to reviews made?
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why? (Justify your answer well!) [Hint: database has NaN values]


Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
### Groups  

Groups should have 4 to 5 people.
You should register your group on **moodle**.

### Submission      

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 13**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/HW1/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/Users/nunoa/Desktop/Aulas/Big Data Management and Modelling/2023/Neo4Jplugins`.
- Change the code bellow accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\anari\OneDrive - NOVAIMS\Desktop\DSAA-BA\Year1\Semester2\Big Data Modelling and Management\HW1\Neo4JPlugins:/plugins" -v "C:\Users\anari\OneDrive - NOVAIMS\Desktop\DSAA-BA\Year1\Semester2\Big Data Modelling and Management\HW1\Neo4JHWData/data:/data" --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist=gds.* neo4j`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

`docker run --name Neo4JHW -p 7474:7474 -p 7687:7687 -d -v "C:\Users\anari\OneDrive - NOVAIMS\Desktop\DSAA-BA\Year1\Semester2\Big Data Modelling and Management\HW1\Neo4JPlugins:/plugins" -v "C:\Users\anari\OneDrive - NOVAIMS\Desktop\DSAA-BA\Year1\Semester2\Big Data Modelling and Management\HW1\Neo4JHWData/data:/data" --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted="gds.*" --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

In [1]:
from neo4j import GraphDatabase
from pprint import pprint

In [2]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [3]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

### All The Functions you'll need to run queries in Neo4J

In [4]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

### Understanding the Database

In [5]:
query = """
        call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


In [6]:
query = """
        CALL db.relationshipTypes();
    """

result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


# Submission

GROUP NUMBER:

**24**

GROUP MEMBERS:

|STUDENT NUMBER|STUDENT NAME|
|---|---|
|Ana Miguel Monteiro|20221645|
|Ana Rita Viseu|20220703|
|Ana Sofia Mendonça|20220678|
|Sara Galguinho|20220682|
|Susana Dias|20220198|

#### 1. How many different countries exist in the database?

In [7]:
# Counting the number of nodes of COUNTRIES label
query = """
        MATCH (c:COUNTRIES)
        RETURN count(c)
    """

result = execute_read(driver, query)
pprint(result)

[<Record count(c)=200>]


There are 200 different countries in the database.

#### 2. Most reviews:  
    A) Which `Beer` has the most reviews?  
    B) Which `Brewery` has the most reviews for its beers?
    C) Which `Country` has the most reviews for its beers? 

In [8]:
# A) Which `Beer` has the most reviews?

# Matching beers to reviews.
# Counting the number of relationships each node of label BEERS has with node label REVIEWS in order to get how many reviews
# each beer has.
# Ordering by the number of reviews in descending order.
# Showing the first 2 records in case there's more than one beer with the most reviews.

query = """
        MATCH (b:BEERS)-[]->(r:REVIEWS)
        RETURN b.name, count(*) AS number_reviews
        ORDER BY number_reviews DESC
        LIMIT 2
    """ 

result = execute_read(driver, query)
pprint(result)

[<Record b.name='IPA' number_reviews=31387>,
 <Record b.name='Porter' number_reviews=21069>]


The Beer with the most reviews is "IPA", with 31387 reviews.

In [9]:
# B) Which `Brewery` has the most reviews for its beers?

# Using a 3-node path to connect breweries to reviews.
# Counting how many total reviews the beers from each brewery have and ordering them in descending order.
# The function id() retrieves the unique id of the nodes.

query = """
        MATCH (br:BREWERIES)-[]->(b:BEERS)-[]->(r:REVIEWS)
        RETURN id(br), br.name, count(*) AS number_reviews
        ORDER BY number_reviews DESC
        LIMIT 2
    """

result = execute_read(driver, query)
pprint(result)

[<Record id(br)=25463 br.name='Sierra Nevada Brewing Co.' number_reviews=175161>,
 <Record id(br)=55933 br.name='Stone Brewing' number_reviews=173282>]


The Brewery with the most reviews for its beers is "Sierra Nevada Brewing Co.", with 175161 reviews.

In [10]:
# C) Which `Country` has the most reviews for its beers?

# Using a 5-node path to connect countries to reviews.
# Counting how many total reviews the beers from each country have and ordering them in descending order.

query = """
        MATCH (c:COUNTRIES)<-[]-(ci:CITIES)<-[]-(br:BREWERIES)-[]->(b:BEERS)-[]->(r:REVIEWS)
        RETURN id(c), c.name, count(*) AS number_reviews
        ORDER BY number_reviews DESC
        LIMIT 2
    """

result = execute_read(driver, query)
pprint(result)

[<Record id(c)=1 c.name='US' number_reviews=7675804>,
 <Record id(c)=2 c.name='GB' number_reviews=727565>]


The Country with the most reviews for its beers is United States, with 7675804 reviews.

#### 3. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [11]:
# Using a 5-node path to connect user "CTJman" to other users (considering users are connected by their reviews to beers).
# Counting the total number of reviews of the same beers (beers with the same id) each user has with "CTJman" and ordering by
# that number in descending order.

query = """
        MATCH (u1:USER{name:'CTJman'})<-[]-(r1:REVIEWS)<-[]-(b:BEERS)-[]->(r2:REVIEWS)-[]->(u2:USER)
        WHERE u2 <> u1
        RETURN id(u2), u2.name, count(id(b)) AS shared_reviews
        ORDER BY shared_reviews DESC
        LIMIT 2
    """

result = execute_read(driver, query)
pprint(result)

[<Record id(u2)=9494459 u2.name='acurtis' shared_reviews=1428>,
 <Record id(u2)=9494682 u2.name='Texasfan549' shared_reviews=1257>]


The user that has the most shared reviews (reviews of the same beers) with "CTJman" is the user "acurtis", with 1428 shared reviews.

#### 4. Which Portuguese brewery has the most beers?


In [12]:
# Selecting Portugal using its code ("PT"), and connecting it to BEERS through CITIES and BREWERIES (4-node path).
# Counting the total number of beers each portuguese brewery has.
# Ordering by number of beers in descending order.

query = """
        MATCH (c:COUNTRIES{name:"PT"})<-[]-(l:CITIES)<-[]-(br:BREWERIES)-[]->(b:BEERS)
        RETURN id(br), br.name as Brewery, count(*) as number_beers
        ORDER BY number_beers DESC
        LIMIT 2
    """

result = execute_read(driver, query)
pprint(result)

[<Record id(br)=27838 Brewery='Dois Corvos Cervejeira' number_beers=40>,
 <Record id(br)=29296 Brewery='Unicer União Cervejeira S.A.' number_beers=22>]


The Portuguese brewery that has the most beers is "Dois Corvos Cervejeira", with 40 different beers.

#### 5. From those beers (the ones produced in the brewery from the previous question), which has the most reviews?


In [13]:
# Using a 3-node path to connect the node of label "Dois Corvos Cervejeira" brewery to Reviews.
# Counting how many total Reviews each beer from the "Dois Corvos Cervejeira" brewery has.
# Ordering by the number of reviews in descending order.

query = """
        MATCH (br:BREWERIES{name:"Dois Corvos Cervejeira"})-[]->(beer:BEERS)-[]->(r:REVIEWS)
        RETURN beer.id, beer.name, count(*) as number_reviews
        ORDER BY number_reviews DESC
        LIMIT 2
        """


result = execute_read(driver, query)
pprint(result)

[<Record beer.id='216709' beer.name='Finisterra' number_reviews=10>,
 <Record beer.id='216706' beer.name='Matiné' number_reviews=6>]


The beer from the brewery "Dois Corvos Cervejeira" that has the most reviews is "Finisterra", with 10 reviews.

#### 6. On average how many different beer styles does each brewery produce?

In [14]:
# Using a 3-node path to connect the node label BREWERIES to REVIEWS through BEERS.
# Counting, using the DISTINCT operator how many different styles of beers each brewery produces. 
# Getting the rounded average of how many different beer styles each brewery produces to 1 decimal place.

query = """
        MATCH (br:BREWERIES)-[]->(b:BEERS)-[]->(s:STYLE)
        WITH br, count(DISTINCT s.name) as number_styles
        RETURN round(AVG(number_styles),1)
        """

result = execute_read(driver, query)
pprint(result)

[<Record round(AVG(number_styles),1)=10.6>]


On average, each brewery produces 11 different beer styles.

#### 7. Which brewery produces the strongest beers according to ABV?

In [15]:
# Using a 2-node path to connect the node label BREWERIES to BEERS.
# Since there are missing values that can be actual null or "nan" values as a string, we use the IS NOT NULL 
# operator and the toFloat() function. This function will convert the "abv" property key values to a Float and if they
# cannot be converted to a float, the function will return null.
# After converting the abv property to a float, we filter the beers that are not null.
# Then, we get the beer with maximum ABV in each brewery using the max() function.
# Finally, we order the breweries by maximum ABV in descending order.

query = """
        MATCH (br:BREWERIES)-[]->(b:BEERS) 
        WHERE toFloat(b.abv) IS NOT NULL
        RETURN id(br), br.name AS breweryName, max(toFloat(b.abv)) AS maxAbv
        ORDER BY maxAbv DESC
        LIMIT 3
        """

result = execute_read(driver, query)
pprint(result)

[<Record id(br)=16084 breweryName='Avondale Brewing Co.' maxAbv=100.0>,
 <Record id(br)=31123 breweryName='1648 Brewing Company Ltd' maxAbv=100.0>,
 <Record id(br)=59088 breweryName='Morgan Territory Brewing' maxAbv=80.0>]


According to ABV, "Avondale Brewing Co." and "1648 Brewing Company Ltd" produce the strongest beers.

#### 8. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [16]:
# Using a 3-node path to match Style to Reviews through Beers.
# Coverting the property keys "smell" and "look" to a float and filtering the ones that are not null (because there are missing
# values).
# Getting the average score in "smell" and "look" each beer style has and ordering them in descending order.

query = """
        MATCH (s:STYLE)<-[]-(b:BEERS)-[]->(r:REVIEWS)
        WHERE toFloat(r.smell) IS NOT NULL AND toFloat(r.look) IS NOT NULL
        RETURN id(s), s.name AS beer_style, round(AVG(toFloat(r.smell)),2) AS avg_smell, round(AVG(toFloat(r.look)),2) AS avg_look
        ORDER BY avg_smell DESC, avg_look DESC
        LIMIT 2
    """

result = execute_read(driver, query)
pprint(result)

[<Record id(s)=9494316 beer_style='New England IPA' avg_smell=4.41 avg_look=4.38>,
 <Record id(s)=9494216 beer_style='American Imperial Stout' avg_smell=4.26 avg_look=4.29>]


If a person typically enjoys a beer due to its aroma and appearance, they should try the beer style "New England IPA" because it's the beer style that has the best average score in smell and look.

#### 9. Using Graph Algorithms answer **one** of the following questions:
    1. Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?
    2. Which beer is the most influential when considering beers are conected by users who review them?
    3. Users are connected together by their reviews to beers, taking into consideration the "overall" score they review as a weight, how many communities are formed from these relationships? How many users has the biggest community?

In [17]:
# QUESTION 9.1: Which two countries are most similiar when it comes to their **top 10** most produced Beer styles?

# Step 0 - Clearing graph
# Dropping any previously created graph with the name "question_9_1" (graph names need to be unique).

try:
    query = """
            CALL gds.graph.drop('question_9_1') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='question_9_1'>]


In [18]:
# Step 1 - Creating a graph

# Projecting the nodes COUNTRIES and STYLE.
# Connecting Countries to Style through a 5-node path.
# Counting the frequency of each beer style (stored in variable 'freq').
# Ordering the beer styles by their frequency in descending order.
# Creating a list containing the top 10 beer styles, using COLLECT.
# Using the UNWIND clause to expand the previous list into a sequence of rows.
# Returning pairs of nodes representing the country and the top 10 beer styles produced in that country.

try:
    query = """
        CALL gds.graph.project.cypher(
            'question_9_1',
            "MATCH (c:COUNTRIES) RETURN id(c) AS id UNION ALL MATCH (s:STYLE) RETURN id(s) AS id",
            "MATCH (c:COUNTRIES)<-[]-(city:CITIES)<-[]-(br:BREWERIES)-[]->(b:BEERS)-[]->(s:STYLE)
            WITH c, s, COUNT(*) AS freq
            ORDER BY freq DESC
            WITH c, COLLECT(s)[..10] AS top10
            UNWIND top10 AS top10_styles
            RETURN id(c) AS source, id(top10_styles) AS target
            "
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (c:COUNTRIES) RETURN id(c) AS id UNION ALL MATCH (s:STYLE) RETURN id(s) AS id' relationshipQuery='MATCH (c:COUNTRIES)<-[]-(city:CITIES)<-[]-(br:BREWERIES)-[]->(b:BEERS)-[]->(s:STYLE)\n            WITH c, s, COUNT(*) AS freq\n            ORDER BY freq DESC\n            WITH c, COLLECT(s)[..10] AS top10\n            UNWIND top10 AS top10_styles\n            RETURN id(c) AS source, id(top10_styles) AS target' graphName='question_9_1' nodeCount=313 relationshipCount=1418 projectMillis=10261>]


In [19]:
# Step 2 - Running the algorithm

# Using the Node Similarity algorithm to calculate the similarity between each pair of country nodes.
# Returning the pairs of country nodes and their respective similarity.
# Ordering by similarity in descending order to get the countries with the highest similarity.

try:
    query = """
        CALL gds.nodeSimilarity.stream('question_9_1')
            YIELD node1, node2, similarity
            RETURN gds.util.asNode(node1).name AS Country1, gds.util.asNode(node2).name AS Country2, similarity AS similarity
            ORDER BY similarity DESC
            LIMIT 1
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record Country1='AU' Country2='NZ' similarity=1.0>]


The two countries that are most similiar when it comes to their top 10 most produced beer styles are Australia and New Zealand.

#### 10. Using Graph Algorithms answer **one** of the following questions:
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    2. Which user is the most influential when it comes to reviews made?

In [20]:
# QUESTION 10.2: Which user is the most influential when it comes to reviews made?

# Step 0 - Clearing graph
# Dropping any previously created graph with the name "question_10_2" (graph names need to be unique).

try:
    query = """
            CALL gds.graph.drop('question_10_2') YIELD graphName;
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record graphName='question_10_2'>]


In [21]:
# Step 1 - Creating a graph

# Projecting the nodes USER and REVIEWS.
# Connecting Users to Reviews.
# Returning the id of the review as the source and the id of the user as the target

try:
    query = """
        CALL gds.graph.project.cypher(
            'question_10_2',
            "MATCH (u:USER) RETURN id(u) AS id UNION ALL MATCH (r:REVIEWS) RETURN id(r) AS id",
            "MATCH (u:USER)<-[]-(r:REVIEWS) RETURN id(r) AS source, id(u) AS target"
            )
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (u:USER) RETURN id(u) AS id UNION ALL MATCH (r:REVIEWS) RETURN id(r) AS id' relationshipQuery='MATCH (u:USER)<-[]-(r:REVIEWS) RETURN id(r) AS source, id(u) AS target' graphName='question_10_2' nodeCount=9238063 relationshipCount=9073109 projectMillis=53793>]


In [22]:
# Step 2 - Running the algorithm

# Using the PageRank algorithm to calculate the score for each node.
# Returning the user and their respective score.
# Ordering the users by their score in descending order.

try:
    query = """
        CALL gds.pageRank.stream('question_10_2')
            YIELD nodeId, score
            RETURN gds.util.asNode(nodeId).name AS user, score
            ORDER BY score DESC
            LIMIT 1
        """

    result = execute_read(driver, query)

    pprint(result)
except Exception as e:
    pprint(e)

[<Record user='Sammy' score=1759.2675000004458>]


The most influential user when it comes to reviews made is "Sammy".

#### 11. If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [23]:
# To answer this question, we considered the reviews made to each beer.
# Because older reviews can be outdated, we only consider the reviews starting from 2013.
# Then we considered the average sum of taste, feel, smell and look each beer has, as well as the average score and overall.

# Steps:
# 1. Connecting Beers to Reviews.
# 2. Because there are missing values in many of the properties of node label REVIEWS, first we convert the properties "score",
#    "overall", "taste", "feel", "smell" and "look" to floats using the toFloat function. Then we filter the reviews where these
#    properties aren't null values using the IS NOT NULL operator.
#    We also filter the reviews without null values in "date" and where the year is higher than 2013 (we extract the year of the
#    review using the substring function). We only considered the reviews posted after 2013 because they would be more relevant
#    than reviews posted a long time ago.
#    Finally, we filter the beers that are still produced (b.retired='f') and that are available all year round.
# 3. Calculating the rounded average of "score", "overall", average of "taste", "feel", "smell" and "look" and storing it in
#    the variable "average characteristics".
# 4. Ordering the beers by their average charactheristics, average score and average overall in descending order.

query = """
        MATCH (b:BEERS)-[]->(r:REVIEWS)
        
        WHERE toFloat(r.score) IS NOT NULL AND toFloat(r.overall) IS NOT NULL AND toFloat(r.taste) IS NOT NULL AND 
        toFloat(r.feel) IS NOT NULL AND toFloat(r.smell) IS NOT NULL AND toFloat(r.look) IS NOT NULL AND r.date IS NOT NULL
        AND substring(r.date, 0, 4)>'2013' AND b.retired='f' AND b.availability CONTAINS 'Year-round'
        
        RETURN b.name AS beer_name, AVG(toFloat(r.score)) AS avg_score, AVG(toFloat(r.overall)) AS avg_overall,
        AVG((toFloat(r.taste) + toFloat(r.feel) + toFloat(r.smell) + toFloat(r.look))/4) AS avg_characteristics, 
        substring(r.date, 0, 4) AS year
        
        ORDER BY avg_characteristics DESC, avg_score DESC, avg_overall DESC
        LIMIT 3
    """

result = execute_read(driver, query)
pprint(result)

[<Record beer_name='Dreadnought' avg_score=5.0 avg_overall=5.0 avg_characteristics=5.0 year='2014'>,
 <Record beer_name='Crimson King' avg_score=5.0 avg_overall=5.0 avg_characteristics=5.0 year='2016'>,
 <Record beer_name='Skullville Jalapeño Pale Ale' avg_score=5.0 avg_overall=5.0 avg_characteristics=5.0 year='2018'>]


If we had to pick 3 beers to recommend using only this database, we would pick "Atomic Pale Ale", "Crimson King", and "Dreadnought".